In [1]:
!pip install pykrx
# https://github.com/sharebook-kr/pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 30.1 MB/s eta 0:00:00


In [4]:
import math
import time
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from pykrx import stock
import pandas as pd
import random

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

<ipython-input-5-4bc53803c362>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [37]:
def get_ohlc(start:datetime, end:datetime, tickernumber:str):  # 시작날짜부터 끝날짜까지 종목 df 받아오기
    df = stock.get_market_ohlcv_by_date(start, end, tickernumber)
    if len(df) == 0:
        return []
    # df.columns = ['Open', 'High', 'Low', 'Close',
                #   'Volume', 'Volume_price', 'rate']
    return df

In [38]:
start_date = datetime(2021,1,1)
end_date = datetime(2023,6,30)
tickernumber = '005930' # 삼성전자 종목 코드
samsung_df = get_ohlc(start_date, end_date, tickernumber)

In [9]:
def get_today_count(df, date:datetime):  # df.index에서 당일 날짜 카운트하기
    try:
        index_list = list(df.index)
        today_count = index_list.index(date)
        return today_count
    except:
        print("요일 확인할것")
        return -1

In [11]:
today = datetime(2023,4,1)
today_index = get_today_count(samsung_df, today)
print(today_index) # 4월 1일은 주식시장이 안 열었기 때문에 -1 반환

요일 확인할것
-1


In [15]:
today = datetime(2023,4,3)
today_index = get_today_count(samsung_df, today)
print(today_index) #
print(samsung_df[today_index:today_index+5]) # 4월 3일부터 5일치 삼성 데이터

556
               시가     고가     저가     종가       거래량           거래대금   등락률
날짜                                                                   
2023-04-03  64000  64000  63000  63100  11973133  757289145500  -1.41
2023-04-04  63400  63800  62800  63600  11120514  706088571400  0.79 
2023-04-05  63700  64000  63400  63900  9176149   584822515500  0.47 
2023-04-06  63500  63600  62300  62300  14992747  940919458200  -2.50
2023-04-07  63800  65200  63800  65000  27476120  1778208943700 4.33 


In [18]:
def get_high_price(df, key:str): # 고점 구하기
    df = df[df[key] != 0]
    if len(df) == 0:
        return -1,-1
    high_price_index = df.loc[:, [key]].idxmax(axis=0)
    high_price = df._get_value(high_price_index[0], key)
    return high_price, high_price_index[0]


def get_low_price(df, key):  # 저점 구하기
    df = df[df[key] != 0]
    if len(df) == 0:
        return -1,-1
    low_price_index = df.loc[:, [key]].idxmin(axis=0)
    low_price = df._get_value(low_price_index[0], key)
    return low_price, low_price_index[0]

In [20]:
# 삼성의 1년 종가 고점, 저점 구하기
print(get_high_price(samsung_df[-250:], '종가'))
print(get_low_price(samsung_df[-250:], '종가'))
# 1년동안의 고점은 72700 저점은 52600

(72700, Timestamp('2023-06-28 00:00:00'))
(52600, Timestamp('2022-09-29 00:00:00'))


In [ ]:
# 종목들의 가격 일봉 데이터를 구하고 1년 고점과 저점 구하는 예시
end_date = datetime(2023,7,9)
start_date = start_date - timedelta(365) # 시작날짜부터 365일 전
sample = pd.read_csv('sample_submission.csv')
for sample_num, tickernumber in enumerate(sample['종목코드'].str.replace('A','')):
    time.sleep(0.2) # 너무 빠르게 요청하면 차단당함
    ticker_df = get_ohlc(start_date, end_date, tickernumber)
    high_price = get_high_price(ticker_df, '고가')[0]
    low_price = get_low_price(ticker_df, '저가')[0]
    print(sample_num, tickernumber)
    print(ticker_df.head(5))
    print(f'고가 = {high_price}, 저가 = {low_price}')
    if sample_num == 5:
        break


In [ ]:
# 7월 3일 종가에 사서 다음날 종가에 팔았을 경우 수익률
end_date = datetime(2023,7,9)
start_date = start_date - timedelta(365) # 시작날짜부터 365일 전
today = datetime(2023,7,3)
sample = pd.read_csv('sample_submission.csv')
for sample_num, tickernumber in enumerate(sample['종목코드'].str.replace('A','')):
    time.sleep(0.2) # 너무 빠르게 요청하면 차단당함
    ticker_df = get_ohlc(start_date, end_date, tickernumber)
    today_count = get_today_count(ticker_df, today)
    if today_count == -1:
        print('상장폐지')
        continue
    d0_Close = ticker_df.iloc[today_count]['종가']
    d1_Close = ticker_df.iloc[today_count+1]['종가']
    earn = d1_Close / d0_Close
    earn = round(earn-1, 2)
    print(d1_Close, d0_Close)
    print(f'수익률 : {earn}')
    if sample_num == 10:
        break


In [65]:
def get_sharpe(df):
    df['dpc'] = (df['종가'] - df['종가'].shift(1)) / df['종가'].shift(1) * 100
    df = df.dropna()
    return (df['dpc'].sum() - 3.5) / df['dpc'].std()

In [ ]:
#어떤 종목을 매일 종가에 사서 다음날 종가에 파는 행위를 15일동안 했을 때 샤프지수
end_date = datetime(2023,6,21)
start_date = datetime(2023,5,31)
sample = pd.read_csv('sample_submission.csv')
result_df = pd.DataFrame()
result_num = 0
for sample_num, tickernumber in enumerate(sample['종목코드'].str.replace('A','')):
    time.sleep(0.2) # 너무 빠르게 요청하면 차단당함
    ticker_df = get_ohlc(start_date, end_date, tickernumber)
    tickername = stock.get_market_ticker_name(tickernumber)
    sharpe = get_sharpe(ticker_df)
    print(sample_num, tickernumber, tickername, sharpe)
    result_df.loc[result_num, '종목코드'] = tickernumber
    result_df.loc[result_num, '종목명'] = tickername
    result_df.loc[result_num, '매수 샤프지수'] = sharpe
    result_df.loc[result_num, '매도 샤프지수'] = -sharpe
    result_num += 1


0 000020 동화약품 5.192399232308422
1 000040 KR모터스 -14.458407547778064
2 000050 경방 -10.772080291998765
3 000070 삼양홀딩스 1.7126422726230488
4 000080 하이트진로 -8.033740541657984
5 000100 유한양행 -3.9889839156325166
6 000120 CJ대한통운 -8.276936124125132
7 000140 하이트진로홀딩스 -9.125054902458153
8 000150 두산 -0.892402228293815
9 000180 성창기업지주 1.620152688896821
10 000210 DL -3.850637528577267
11 000220 유유제약 -8.79767356925249
12 000230 일동홀딩스 -12.194133534077727
13 000240 한국앤컴퍼니 -3.488188152149267
14 000250 삼천당제약 -3.2544687500761547
15 000270 기아 -2.922281698619215
16 000300 대유플러스 4.768942110736431
17 000320 노루홀딩스 -0.5934657044696918
18 000370 한화손해보험 -10.265065889071236
19 000390 삼화페인트 0.8089442490859072
20 000400 롯데손해보험 -1.5737260274642837
21 000430 대원강업 -1.6308185165983535
22 000440 중앙에너비스 -3.5191780525642553
23 000480 조선내화 6.274360766712768
24 000490 대동 -0.11566221573856157
25 000500 가온전선 2.31643411035926
26 000520 삼일제약 -4.132953231123141
27 000540 흥국화재 -4.966855635325073
28 000590 CS홀딩스 -3.361025127143665
29 0